In [ ]:
#Libraries for data manipulation
import pandas as pd
import numpy as np

#Library for splitting data
from sklearn.model_selection import train_test_split 

#Libraries for hyperparameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
#Dictionary with the best hyperparameters for each model
#hyperparameters = {
    #'randomForest' : ,
    #'xgboost' : ,
    #'decissionTreClassifier' : ,
    #'knn' : ,
    #'svm': ,
    #'lgb' : ,
#}

In [ ]:

#Function thar returns the best hyperparameters for each model
#def getHyperparameters(model):
    #return hyperparameters['model']

In [ ]:
trainCSV = pd.read_csv('./forHiper')
trainCSV.head()

In [ ]:
trainCSV = trainCSV.iloc[:, 4:]
x, y = trainCSV.iloc[:,1:], trainCSV.iloc[:,0]  #X tiene que tener todos los features distintos al target
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [ ]:
#Searchs for the best hyperparameters using Grid Search
def findHyperparametersGS(xTrain, yTrain, model, param_grid, cv, vb, nj):
    
    grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = 'roc_auc', cv = cv, verbose = vb, n_jobs = nj)
    grid.fit(xTrain, yTrain)
    return grid.best_params_

In [ ]:
#Searchs for the best hyperparameters using Randomized Search
def findHyperparametersRS(x, y, model, param_grid, n_iter, vb, nj):
    xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 0.25, random_state = 42)
    grid = RandomizedSearchCV(estimator = model, params_grid, n_iter = n_iter, scoring='roc_auc', verbose = vb, n_jobs = nj)
    grid.fit(xTrain, yTrain)
    return grid.best_params_

## Random forest

https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators' : [100, 300, 500, 800, 1200],
    'max_depth' : [5, 8, 15, 25, 30],
    'min_samples_split' : [2, 5, 10, 15, 100],
    'min_samples_leaf' : [1, 2, 5, 10],
    'criterion' : ['gini','entropy']
}

RC = RandomForestClassifier()
bestParametersRF = findHyperparametersGS(xTrain, yTrain, RC, param_grid, 3, 1, -1) #bestParametersRF
bestParametersRF

## XGBoost

https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost

In [ ]:
import xgboost as xgb #conda install -c conda-forge xgboost 

param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
}
XGB = XGBClassifier()
bestParametersXGB = findHyperparametersGS(xTrain, yTrain, XGB, param_grid, skf.split(xTrain, yTrain), 3, 4)
bestParametersXGB

## Decission Tree Classifier

https://www.kaggle.com/mayu0116/hyper-parameters-tuning-of-dtree-rf-svm-knn

In [ ]:
from sklearn import tree

param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
          'min_samples_split': [2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
          'min_samples_leaf':[1,2,3,4,5,6,7,8,9,10,11],
          'random_state':[123]}

DTC = tree.DecisionTreeClassifier(min_samples_split = 10)
bestParameteresDTC = findHyperparametersGS(xTrain, yTrain, DTC, param_grid, 3, 1, -1)
bestParameteresDTC

## KNN

https://www.kaggle.com/mayu0116/hyper-parameters-tuning-of-dtree-rf-svm-knn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'n_neighbors':[5,6,7,8,9,10],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance'],
          'algorithm':['auto', 'ball_tree','kd_tree','brute'],
          'n_jobs':[-1]}

KNN = KNeighborsClassifier()
bestParameteresKNN = findHyperparametersGS(xTrain, yTrain, KNN, param_grid, 3, 1, 1)
bestParameteresKNN

## Support Vector Machine

https://www.kaggle.com/mayu0116/hyper-parameters-tuning-of-dtree-rf-svm-knn

In [ ]:
from sklearn import svm

param_grid = {'C': [6,7,8,9,10,11,12], 
          'kernel': ['linear','rbf']}

SVC = svm.SVC()
bestParameteresSVC = findHyperparametersGS(xTrain, yTrain, SVC, param_grid, 3, 1, -1)
bestParameteresSVC

## Light GBM

https://mlfromscratch.com/gridsearch-keras-sklearn/#/

In [ ]:
import lightgbm as lgb

param_grid = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'num_leaves': [50, 100, 200],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}

LGBM = lgb.LGBMClassifier()

bestParameteresLGBM = algorithm_pipeline(xTrain, yTrain, LGBM, param_grid, 5, 2, -1)
bestParameteresLGBM